In [1]:
addpath ("files")
setenv('GNUTERM','qt')

In [3]:
function  [gfx] = gradiente(f, x, h = 1.e-08)

%{
    Aproximacion del gradiente por diferencias hacia adelante 
    de una funcion  de R^n a R.
    
    Input:
        f   .- apuntador a la función
        x   .- vector columna de dimension n.

    Output:

        gfx .- vector columna de dimension n, 
                 es la aproximacion al gradiente en x.
%}

    n = length(x);
    
    fx = f(x);                            
    xt = x;
    
    for i = 1:n
        xt(i) = xt(i) + h;
        fxh = f(xt);
        gfx(i) = (fxh - fx)/h;
        xt(i) = x(i);
    end
end

In [4]:
function [H] = hessian(f,x, h = 1.e-08)

    n = length(x);
    H = zeros(n);
    fx = f(x);

    for i = 1:n
        x1 = x;
        x1(i) = x1(i) + h;
        f1 = f(x1);
        for j = i:n
            x2 = x;
            x2(j) = x2(j) + h;
            f2 = f(x2);

            x3 = x1;
            x3(j) = x3(j) + h;
            f3 = f(x3);
            
            H(i,j) = (f3 - f2 - f1 + fx)/(h^2);
            H(j,i) = H(i,j);
        end
    end
end

In [ ]:

function [W,x,iter] = metodoblinterpol2(f, x, method='newton', line_search='quadratic',
                                        tol=1.e-08, maxiter = 250, maxjter = 20, c1 = 1.e-04)
%{
    Método de búsqueda de línea con la primer condición de Wolfe
    usando máximo descenso.
    usando los métodos de descenso:
        1. Máximo descenso
        2. Método de Newton
    Input:
        
        fname.- cadena de caracteres con el nombre de la función a minimizar.
        x    .- vector n-dimensional.
    
    Output:
        x    .- vector n-dimensional con la aproximación al mínimo local.
        iter .- contador con el número final de iteraciones externas.
%}
    
    iter = 0;        
    p = 1; 

    g  = gradiente2(@f,x);
    W  = [x]; 

    while ( norm(g) > tol && iter < maxiter)

        if (strcmp(method,'max'))
            p = -g;
        end

        if (strcmp(method,'newton'))
            H = hessian2(f,x);
            p = -H\g; 
        end

        %-----------------------------------------
        %            Búsqueda de línea
        
        alfa  = 1;              
        xt = x + alfa*p;        
        f  = f(x);
        f1 = f(xt);
        s  = p'*g;            
        jter = 0;             

        if (abs(s) < 1.e-06)
            disp('No existe suficiente descenso  ')
            disp(sprintf('%2.0f  %2.10f',iter, s   ))
            iter = maxiter; 
        end

        while( (f1>f+alfa*c1*s) && jter < maxjter)  % búsqueda de línea
        
            if (strcmp(line_search,'linear'))   
                alfa = alfa/2;
                xt = x + alfa*p;
                f1 = feval(fname, xt);  
                jter = jter +1;
            end
            
            if (strcmp(line_search,'quadratic'))
                d2 = f1 - f - s;
                alfa = -(s)/(2*d2);       
                f1 = feval(fname, x + alfa*p);
                jter = jter +1;    
            end

        end   
        
        %          Fin de búsqueda de línea
        %------------------------------------------
        
        
        %------------------------------------------
        %                Graficación

        if (plot_iterations == true)
        
            t = linspace(0,1,50)';
            ft = zeros(50,1); rt = zeros(50,1);
            for k = 1:50        
                ft(k) = feval(fname, x+t(k)*p);  % función 
                rt(k) = fx + t(k)*(c1*p'*g);
            end

            fx = feval(fname,x+alfa*p);

            plot(t,ft,'--b',t,rt,'--m',alfa,fx,'dr', 'LineWidth',3)
            title('Gráfica de búsqueda de línea','Fontsize',16)
            xlabel('EJE  T','Fontsize',16)
            ylabel(' f(x + tp)','Fontsize',16  )
            legend('f(x)','recta','punto')
            pause
        end

        %              Fin de graficación
        %-----------------------------------------
          
        W = [W x];
        x = x + alfa*p;  
        fx = feval(fname,x); 
        g = gradiente(fname,x);
        iter = iter + 1;
        
    end      
end